# Preparation

In [ ]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [ ]:
def pull_model(local_llm):
    !ollama pull local_llm

In [ ]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [ ]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Dati

https://www.kaggle.com/datasets/waalbannyantudre/hate-speech-detection-curated-dataset

https://www.sciencedirect.com/science/article/pii/S2352340922010356

In [ ]:
# todo #all'incirca 7.000 testi di test

df = pd.read_csv("/kaggle/input/hate-speech-detection-curated-dataset/HateSpeechDatasetBalanced.csv")

print(len(df))

#label_0 = df[df["Label"]==0] #no hate speech
#label_1 = df[df["Label"]==1] #hate speech
#print(f"Label con valore 0: {len(label_0)}")
#print(f"Label con valore 1: {len(label_1)}")

# Campiona l'1% del DataFrame
sampled_df = df.sample(frac=0.01, random_state=42)

x_test = sampled_df["Content"]
y_test = sampled_df["Label"]

print(len(x_test))
print(len(y_test))

label_0 = sampled_df[sampled_df["Label"]==0] #no hate speech
label_1 = sampled_df[sampled_df["Label"]==1] #hate speech
print(f"Label con valore 0: {len(label_0)}")
print(f"Label con valore 1: {len(label_1)}")

# Hate speech

In [ ]:
model = "llama3.1"

start_ollama()
pull_model(model)
start_model(model)

In [ ]:
# Examples for few shot:

examples = ["""
            Example 1: \n
            Text: We need to keep our culture pure. \n
            Answer: yes \n
            
            Example 2: \n
            Text: It’s important to respect other people's choices. \n
            Answer: no \n
            
            Example 3: \n
            Text: They're just different, and that's not a bad thing, but they shouldn't try to change us. \n
            Answer: yes \n
            
            Example 4: \n
            Text: I think the weather is nice today. \n
            Answer: no \n
            
            Example 5: \n
            Text: Is it really hate if it's true?. \n
            Answer: yes \n
            
            Example 6: \n
            Text: Have a great day!. \n
            Answer: no \n
            
            Example 7: \n
            Text: You are [insult]. \n
            Answer: yes \n
            
            Example 8: \n
            Text: We should celebrate diversity.. \n
            Answer: no \n
            """]

In [40]:
prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an helpful assistant who has to detect the presence of hate speech.
            Hate speech is speech that attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. 
            You have to answer "yes" if it contains hate speech, or "no" if it doesn't contain hate speech. 
            Give only the answer, NO PREAMBLE, NO EXPLANATIONS. \n
            Here some examples: \n
            
            Example 1: \n
            Text: We need to keep our culture pure. \n
            Answer: yes \n
            
            Example 2: \n
            Text: It’s important to respect other people's choices. \n
            Answer: no \n
            
            Example 3: \n
            Text: They're just different, and that's not a bad thing, but they shouldn't try to change us. \n
            Answer: yes \n
            
            Example 4: \n
            Text: I think the weather is nice today. \n
            Answer: no \n
            
            <|eot_id|><|start_header_id|>user<|end_header_id|> 
            Text: {document}. \n
            Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["document"],
    )

llm = Ollama(model=model, temperature=0)

In [41]:
def hate_speech_detection(prompt, llm):
    hate_speech_detection = prompt | llm | StrOutputParser()
    return hate_speech_detection

In [ ]:
# Label 0: no-hate speech (no answer)
# Label 1: hate speech (yes answer)

def predict(prompt,llm,x_test):
    y_pred = []
    for x in tqdm(x_test):
        answer = hate_speech_detection(prompt,llm).invoke({"document":x})
        if answer.lower() == "yes": y_pred.append(1)
        else: y_pred.append(0)
    return y_pred

y_pred = predict(prompt,llm,x_test)

  0%|          | 0/7261 [00:00<?, ?it/s]time=2024-08-09T16:55:39.300Z level=INFO source=sched.go:710 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-8eeb52dfb3bb9aefdf9d1ef24b3bdbcfbe82238798c4b918278320b6fcef18fe gpu=GPU-2ae44804-7041-be37-1b54-40e3cc9504e8 parallel=4 available=16790978560 required="6.2 GiB"
time=2024-08-09T16:55:39.301Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[15.6 GiB]" memory.required.full="6.2 GiB" memory.required.partial="6.2 GiB" memory.required.kv="1.0 GiB" memory.required.allocations="[6.2 GiB]" memory.weights.total="4.7 GiB" memory.weights.repeating="4.3 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-08-09T16:55:39.302Z level=INFO source=server.go:392 msg="starting llama server" cmd="/tmp/ollama3952069877/runners/cuda_v11/ollama_lla

INFO [main] build info | build=1 commit="1e6f655" tid="139580736086016" timestamp=1723222539
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="139580736086016" timestamp=1723222539 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="37443" tid="139580736086016" timestamp=1723222539


time=2024-08-09T16:55:39.555Z level=INFO source=server.go:626 msg="waiting for server to become available" status="llm server loading model"
llama_model_loader: - kv  24:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  25:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - kv  26:                tokenizer.ggml.eos_token_id u32              = 128009
llama_model_loader: - kv  27:                    tokenizer.chat_template str              = {{- bos_token }}\n{%- if custom_tools ...
llama_model_loader: - kv  28:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   66 tensors
llama_model_loader: - type q4_0:  225 tensors
llama_model_loader: - type q6_K:    1 tensors
llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.7999 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_prin

INFO [main] model loaded | tid="139580736086016" timestamp=1723222542


time=2024-08-09T16:55:43.071Z level=INFO source=server.go:631 msg="llama runner started in 3.77 seconds"
  0%|          | 1/7261 [00:04<9:52:39,  4.90s/it]

...

[GIN] 2024/08/09 - 17:20:41 | 200 |  819.794838ms |       127.0.0.1 | POST     "/api/generate"


 35%|███▌      | 2573/7261 [25:02<45:25,  1.72it/s]

[GIN] 2024/08/09 - 17:20:41 | 200 |  502.824021ms |       127.0.0.1 | POST     "/api/generate"


 35%|███▌      | 2574/7261 [25:03<42:26,  1.84it/s]

[GIN] 2024/08/09 - 17:20:42 | 200 |  438.503332ms |       127.0.0.1 | POST     "/api/generate"


In [ ]:
# Valutazione del modello
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, labels=[0,1], output_dict=True)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

In [ ]:
# Salva risultati

import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)
        

dict_to_write = {"class_report":class_report, "0_0":int(conf_matrix[0][0]), "0_1":int(conf_matrix[0][1]), "1_0":int(conf_matrix[1][0]), "1_1":int(conf_matrix[1][1])}        
write_file("/kaggle/working/test_hate_speech_llama31_four_shot.json", dict_to_write)